# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.151080382998771
GPU (s):
0.11110841200024879
GPU speedup over CPU: 28x


## To access files in google drive we have to connect google colab to google drive

In [ ]:
from google.colab import drive #For using the google drive for data
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np#Array handling
import pandas as pd#Working with dataframes

### Reading the data

In [ ]:
train=pd.read_csv('/content/drive/My Drive/dataset/train.csv')
test=pd.read_csv('/content/drive/My Drive/dataset/test.csv')

In [ ]:
train.head()#Make sure data is read correctly

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [ ]:
test.head()#See data is correctly read

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg


In [ ]:
train['target'].unique() #Check all the dance forms

array(['manipuri', 'bharatanatyam', 'odissi', 'kathakali', 'kathak',
       'sattriya', 'kuchipudi', 'mohiniyattam'], dtype=object)

In [ ]:
'''
We cannot train a model on categorical label so we have to convert it into numerical
class_map --> it is used to conmvert categorical to numerical to perform computation
inverse_map --> it is used to convert the numerical output obtained after computation into categorical form
'''

'\nWe cannot train a model on categorical label so we have to convert it into numerical\nclass_map --> it is used to conmvert categorical to numerical to perform computation\ninverse_map --> it is used to convert the numerical output obtained after computation into categorical form\n'

In [ ]:
class_map={'manipuri':0,'bharatanatyam':1,'odissi':2, 'kathakali':3, 'kathak':4,'sattriya':5, 'kuchipudi':6, 'mohiniyattam':7}
inverse_map={0:'manipuri',1:'bharatanatyam',2:'odissi',3:'kathakali',4: 'kathak',5:'sattriya',6:'kuchipudi',7:'mohiniyattam'}
train['target']=train['target'].map(class_map) #maps the two series of class_map[dance forms,ints] dance_forms->ints

In [ ]:
train.head() #Check above mapping worked or not

,Image,target
0,96.jpg,0
1,163.jpg,1
2,450.jpg,2
3,219.jpg,3
4,455.jpg,2


In [ ]:
'''
List of pretrained Models for image classification are:
1) Xception
2) VGG16
3) VGG19
4) ResNet50
5) InceptionV3
6) InceptionResNetV2
7) MobileNet
8) MobileNetV2
9) DenseNet
10) NASNet
'''

'\nList of pretrained Models for image classification are:\n1) Xception\n2) VGG16\n3) VGG19\n4) ResNet50\n5) InceptionV3\n6) InceptionResNetV2\n7) MobileNet\n8) MobileNetV2\n9) DenseNet\n10) NASNet\n'

In [ ]:
'''
All the pretrained models listed above have their weights trained on ImageNet Data
All the pretrained models have the shape of image as (224,224,3) 
so to perform transfer learning using above pretrained models we have to convert out images to (224,224,3)
'''

'\nAll the pretrained models listed above have their weights trained on ImageNet Data\nAll the pretrained models have the shape of image as (224,224,3) \nso to perform transfer learning using above pretrained models we have to convert out images to (224,224,3)\n'

In [ ]:
img_h,img_w=(224,224) #For using the standard models to make use of transfer learning

In [ ]:
import os#This module provides a portable way of using operating system dependent functionality.
import seaborn as sns #For graphs
import cv2 #it is used for computer vision applications like image processing,video capture analysis and like face and object detection
from tqdm import tqdm #Shows the progess bar of how much a loop is executed or a pipeline is executed.
import matplotlib.pyplot as plt #For plotting.

In [ ]:
'''
1) we have to read the images from the given path
2) resize the image into standard shape
3) and converting the image array into floaating point array
4) and append each array into train_img
5) append each output label into train_label
'''

'\n1) we have to read the images from the given path\n2) resize the image into standard shape\n3) and converting the image array into floaating point array\n4) and append each array into train_img\n5) append each output label into train_label\n'

In [ ]:
train_img=[] #In this list we will have images of required type as we needed
train_label=[] #lables for images 
j=0
path='/content/drive/My Drive/dataset/train'
for i in tqdm(train['Image']):#using tqdm we can get a progress bar showing how much its done
    final_path=os.path.join(path,i) #setting up path
    img=cv2.imread(final_path) #reading the image
    img=cv2.resize(img,(img_h,img_w)) #resizing the image
    img=img.astype('float32') #Converting all the pixels as float32
    train_img.append(img) #Append it to the list.
    train_label.append(train['target'][j]) #similarly append the label
    j=j+1

100%|██████████| 364/364 [00:02<00:00, 131.67it/s]


In [ ]:
test_img=[]
path='/content/drive/My Drive/dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [00:01<00:00, 124.25it/s]


### Splitting train data to train and validation.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(train_img,train_label,test_size=0.05,shuffle=True)

In [ ]:
'''
Data augmentation :- Data augmentation encompasses a wide range of techniques used to generate “new” training samples from the original ones by applying random jitters and perturbations 

simple geometric transforms, such as random:
1) Translations
2) Rotations
4) 3) Changes in scale
5) Shearing
6) Horizontal (and in some cases, vertical) flips

ImageDataGenerator is a method to perform data augmentation

'''

'\nData augmentation :- Data augmentation encompasses a wide range of techniques used to generate “new” training samples from the original ones by applying random jitters and perturbations \n\nsimple geometric transforms, such as random:\n1) Translations\n2) Rotations\n4) 3) Changes in scale\n5) Shearing\n6) Horizontal (and in some cases, vertical) flips\n\nImageDataGenerator is a method to perform data augmentation\n\n'

In [ ]:
'''Rescale is a value by which we will multiply the data before any other processing. 
Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our model to process 
(given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,# divide each input by its std
        rescale=1./255,
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [ ]:
#We don't do any changes for the test data.
test_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen= ImageDataGenerator(rescale=1./255)
train_datagen.fit(x_train)
test_datagen.fit(test_img)
valid_datagen.fit(x_valid)

In [ ]:
''' CNN works internally only with numpy arrays so we have to convert out data into numpy array  

Flatten as the name implies, converts your multidimensional matrices 
(Batch.Size x Img.W x Img.H x Kernel.Size) to a nice single 2-dimensional matrix: (Batch.Size x (Img.W x Img.H x Kernel.Size)). 

During backpropagation it also converts back your delta of size 
(Batch.Size x (Img.W x Img.H x Kernel.Size)) to the original (Batch.Size x Img.W x Img.H x Kernel.Size).

Dense layer is of course the standard fully connected layer.
'''

' CNN works internally only with numpy arrays so we have to convert out data into numpy array  '

In [ ]:
train_img=np.array(train_img)
x_train= np.array(x_train)
x_valid= np.array(x_valid)
y_train= np.array(y_train)
y_valid= np.array(y_valid)
test_img=np.array(test_img)
train_label=np.array(train_label)
print("Shape of training data=",x_train.shape," and shape of labels of training data= ",y_train.shape)
print("Shape of validation data=",x_valid.shape," and shape of labels of validation data= ",y_valid.shape)
print("Shape of test data=",test_img.shape)

Shape of training data= (345, 224, 224, 3)  and shape of labels of training data=  (345,)
Shape of validation data= (19, 224, 224, 3)  and shape of labels of validation data=  (19,)
Shape of test data= (156, 224, 224, 3)


## InceptionResNetV2

In [ ]:
'''
Sequential is used to initialize the neural network.
Convolution2D is used to make the convolutional network that deals with the images.
MaxPooling2D layer is used to add the pooling layers.
Flatten is the function that converts the pooled feature map to a single column that is passed to the fully connected layer.
Dense adds the fully connected layer to the neural network.
'''

In [ ]:
'''Dropout regularization to avoid the overfitting
Include_top lets you select if you want the final dense layers or not.
the convolutional layers work as feature extractors. They identify a series of patterns in the image, 
and each layer can identify more elaborate patterns by seeing patterns of patterns.
the dense layers are capable of interpreting the found patterns in order to classify: this image contains cats, dogs, cars, etc.

So the dence layers or last ouputting layers need to be manullay add if we are doing the feature engineering.
ImageNet is a benchmark dataset containing millions of images and its weights are very good for many of computer vision tasks.
Both input_tenser and input_shape can't be set arguments any one of its must an argument.Both specifies user specicified input size
'''

from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dropout
base_model= InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_h,img_w,3), pooling='avg')

In [ ]:
#make sure this layers won't gets trained.As they almost have a good weights.
base_model.trainable=False
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 111, 111, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_209 (BatchN (None, 111, 111, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 111, 111, 32) 0           batch_normalization_209[0][0]    
________________________________________________________________________________

In [ ]:
'''Flatten -> is the function that converts the pooled feature map to a single column that is passed to the fully connected layer.
Dense -> adds the fully connected layer to the neural network.
Dropout->regularization
BatchNormalization-> sends data(Images) as the batches

Model -> Model groups layers into an object with training and inference features.
Sequential -> Sequential groups a linear stack of layers into a tf.keras.Model.

to_categorical -> Converts a class vector (integers) to binary class matrix.

Conv2D -> This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs
MaxPooling2D -> Downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis

ReduceLROnPlateau -> Reduce learning rate when a metric(accuracy) has stopped improving.
'''
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [ ]:
model.add(Dense(8,activation='softmax'))

In [ ]:
'''
Adam -> Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments
SGD -> Stochastic gradient descent and momentum optimizer.
Adagrad -> Adagrad is an optimizer with parameter-specific learning rates, which are adapted relative to how frequently a parameter gets updated during training
Adadelta -> Adadelta optimization is a stochastic gradient descent method that is based on adaptive learning rate per dimension
RMSprop -> maintain a moving (discounted) average of the square of gradients
           divide gradient by the root of this average
'''

'\nAdam -> Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments\nSGD -> Stochastic gradient descent and momentum optimizer.\nAdagrad -> Adagrad is an optimizer with parameter-specific learning rates, which are adapted relative to how frequently a parameter gets updated during training\nAdadelta -> Adadelta optimization is a stochastic gradient descent method that is based on adaptive learning rate per dimension\nRMSprop -> maintain a moving (discounted) average of the square of gradients\n           divide gradient by the root of this average\n'

In [ ]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)
callbacks = [reduce_learning_rate]

In [ ]:
''''
compile -> Configures the model for training
evaluate -> Returns the loss value & metrics values for the model in test mode
            Computation is done in batches.
evaluate_generator -> Evaluates the model on a data generator.
fit -> Trains the model for a fixed number of epochs (iterations on a dataset).
fit_generator -> Fits the model on data yielded batch-by-batch by a Python generator.
predict -> Generates output predictions for the input samples.
           Computation is done in batches.
summary -> Prints a string summary of the network.
'''

"'\ncompile -> Configures the model for training\nevaluate -> Returns the loss value & metrics values for the model in test mode\n            Computation is done in batches.\nevaluate_generator -> Evaluates the model on a data generator.\nfit -> Trains the model for a fixed number of epochs (iterations on a dataset).\nfit_generator -> Fits the model on data yielded batch-by-batch by a Python generator.\npredict -> Generates output predictions for the input samples.\n           Computation is done in batches.\nsummary -> Prints a string summary of the network.\n"

In [ ]:
model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
flatten_6 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1536)              0         
_________________________________________________________________
batch_normalization_412 (Bat (None, 1536)              6144      
_________________________________________________________________
dense_18 (Dense)             (None, 512)               786944    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_413 (Bat (None, 512)              

In [ ]:
model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=16),epochs=50,callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=16),verbose=1)

Epoch 1/50
22/22 [==============================] - 9s 389ms/step - loss: 2.3884 - accuracy: 0.1971 - val_loss: 1.9726 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 2/50
22/22 [==============================] - 4s 194ms/step - loss: 1.8817 - accuracy: 0.3855 - val_loss: 1.6582 - val_accuracy: 0.4211 - lr: 0.0010
Epoch 3/50
22/22 [==============================] - 4s 194ms/step - loss: 1.7790 - accuracy: 0.4290 - val_loss: 1.3398 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 4/50
22/22 [==============================] - 4s 194ms/step - loss: 1.5867 - accuracy: 0.4580 - val_loss: 1.3927 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 5/50
22/22 [==============================] - 4s 196ms/step - loss: 1.4664 - accuracy: 0.4464 - val_loss: 1.2822 - val_accuracy: 0.5263 - lr: 0.0010
Epoch 6/50
22/22 [==============================] - 4s 198ms/step - loss: 1.4911 - accuracy: 0.4812 - val_loss: 1.1231 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 7/50
22/22 [==============================] - 4s 195ms/step - lo

In [ ]:
label1 = model.predict(test_img)

In [ ]:
labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,bharatanatyam
1,246.jpg,bharatanatyam
2,473.jpg,bharatanatyam
3,485.jpg,bharatanatyam
4,128.jpg,bharatanatyam


In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/inception.csv', index=False)

## ResNet50

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
base_model_2= ResNet50(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')
'''for layer in base_model_2.layers[:-3]:
    layer.trainable=False'''
base_model_2.trainable=False

In [ ]:
    
model_2=Sequential()
model_2.add(base_model_2)

model_2.add(Flatten())
model_2.add(Dropout(0.4))
model_2.add(BatchNormalization())

model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(BatchNormalization())

model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(0.1))
model_2.add(BatchNormalization())


model_2.add(Dense(8,activation='softmax'))

In [ ]:
model_2.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_415 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_21 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_416 (Bat (None, 512)              

In [ ]:
model_2.fit(      train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),epochs=50,callbacks=callbacks,
validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),verbose=1)

Epoch 1/50
11/11 [==============================] - 5s 481ms/step - loss: 2.7221 - accuracy: 0.1391 - val_loss: 1.9050 - val_accuracy: 0.2632 - lr: 0.0010
Epoch 2/50
11/11 [==============================] - 4s 328ms/step - loss: 2.6184 - accuracy: 0.1797 - val_loss: 1.9471 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 3/50
11/11 [==============================] - 4s 335ms/step - loss: 2.4962 - accuracy: 0.1913 - val_loss: 1.8900 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 4/50
11/11 [==============================] - 4s 331ms/step - loss: 2.2947 - accuracy: 0.2377 - val_loss: 1.8331 - val_accuracy: 0.3684 - lr: 0.0010
Epoch 5/50
11/11 [==============================] - 4s 345ms/step - loss: 2.3931 - accuracy: 0.1826 - val_loss: 1.7309 - val_accuracy: 0.4211 - lr: 0.0010
Epoch 6/50
11/11 [==============================] - ETA: 0s - loss: 2.3443 - accuracy: 0.2087
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
11/11 [==============================] - 4s 334ms/

In [ ]:
label2 = model_2.predict(test_img)

In [ ]:
labels = model_2.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,mohiniyattam
1,246.jpg,mohiniyattam
2,473.jpg,mohiniyattam
3,485.jpg,mohiniyattam
4,128.jpg,mohiniyattam


In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/resnet50.csv', index=False)

## VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
base_model_3=VGG19(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

In [ ]:
for layer in base_model_3.layers[:-4]:
    layer.trainable=False

In [ ]:
# vgg19-3
model_3=Sequential()
model_3.add(base_model_3)
model_3.add(Flatten())
#model_3.add(BatchNormalization())
#model_3.add(Dropout(0.2))
model_3.add(Dense(512, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))
model_3.add(Dense(128, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dense(8,activation='softmax'))

model_3.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#model_3.summary()

In [ ]:
model_3.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=16),epochs=30,callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=16),verbose=1)

Epoch 1/30
22/22 [==============================] - 5s 207ms/step - loss: 2.1345 - accuracy: 0.1942 - val_loss: 1.8601 - val_accuracy: 0.3684 - lr: 0.0010
Epoch 2/30
22/22 [==============================] - 4s 198ms/step - loss: 1.7919 - accuracy: 0.2899 - val_loss: 1.2241 - val_accuracy: 0.5263 - lr: 0.0010
Epoch 3/30
22/22 [==============================] - 4s 197ms/step - loss: 1.4760 - accuracy: 0.4464 - val_loss: 0.9595 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 4/30
22/22 [==============================] - 4s 197ms/step - loss: 1.3952 - accuracy: 0.5420 - val_loss: 0.8612 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 5/30
22/22 [==============================] - 4s 200ms/step - loss: 1.1729 - accuracy: 0.6000 - val_loss: 1.3872 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 6/30
22/22 [==============================] - 4s 201ms/step - loss: 1.1628 - accuracy: 0.5855 - val_loss: 0.5524 - val_accuracy: 0.8947 - lr: 0.0010
Epoch 7/30
22/22 [==============================] - 4s 199ms/step - lo

In [ ]:
label3 = model_3.predict(test_img)

In [ ]:
labels = model_3.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,odissi
1,246.jpg,mohiniyattam
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,bharatanatyam


In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/vgg19.csv', index=False)

### Combine above 3 models

In [ ]:
labels=0.8*label3+0.1*label2+0.1*label1
labels

array([[0.0000000e+00, 1.0000000e-01, 8.0000001e-01, ..., 1.7520537e-28,
        2.2553354e-29, 1.0000000e-01],
       [0.0000000e+00, 8.9824152e-01, 0.0000000e+00, ..., 0.0000000e+00,
        1.7584724e-03, 1.0000000e-01],
       [0.0000000e+00, 1.0000000e-01, 8.0000001e-01, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e-01],
       ...,
       [0.0000000e+00, 1.0000000e-01, 8.0000001e-01, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e-01],
       [0.0000000e+00, 1.0000000e-01, 8.0000001e-01, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e-01],
       [0.0000000e+00, 1.0000000e-01, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e-01]], dtype=float32)

In [ ]:
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()
submission.to_csv('/content/drive/My Drive/dataset/combine.csv', index=False)

## Stacking of above 3 models

In [ ]:
'''
Input -> it is used to instantiate a Keras tensor.
          A Keras tensor is a tensor object from the underlying backend (Theano or TensorFlow),
          which we augment with certain attributes that allow us to build a Keras model just by 
          knowing the inputs and outputs of the model.
Concatenate layer -> It takes as input a list of tensors, all of the same shape except for the concatenation axis,
                     and returns a single tensor that is the concatenation of all inputs.
concatenate -> Functional interface to the Concatenate layer.
get_layer -> Retrieves a layer based on either its name (unique) or index
'''

SyntaxError: ignored

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
def stacking_ensemble(members,input_shape,n_classes):
  commonInput = Input(shape=input_shape)
  out=[]

  for model in members:
    #model._name= model._name+"test"+ str(members.index(model)+1)
    model._name= model.get_layer(index = 0)._name +"-test"+ str(members.index(model)+1)
    out.append(model(commonInput))

  modeltmp = concatenate(out,axis=-1)
  modeltmp = Dense(32, activation='relu')(modeltmp)
  modeltmp = Dense(16, activation='relu')(modeltmp)
  modeltmp = Dense(n_classes, activation='softmax')(modeltmp)
  stacked_model = Model(commonInput,modeltmp)
  stacked_model.compile( loss='categorical_crossentropy',optimizer= 'adam', metrics=['accuracy'])

  return stacked_model

In [ ]:
members=[model,model_2,model_3]

In [ ]:
stacked_model= stacking_ensemble(members,(img_h,img_w,3),8)
stacked_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
inception_resnet_v2-test1 (Sequ (None, 8)            55199080    input_12[0][0]                   
__________________________________________________________________________________________________
resnet50-test2 (Sequential)     (None, 8)            24714248    input_12[0][0]                   
__________________________________________________________________________________________________
vgg19-test3 (Sequential)        (None, 8)            20353736    input_12[0][0]                   
______________________________________________________________________________________________

In [ ]:
stacked_model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),
                    epochs=50,
          callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),
          verbose=1
             )

Epoch 1/50
11/11 [==============================] - 17s 2s/step - loss: 2.0634 - accuracy: 0.1478 - val_loss: 2.0883 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/50
11/11 [==============================] - 5s 493ms/step - loss: 2.0027 - accuracy: 0.2841 - val_loss: 2.0271 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 3/50
11/11 [==============================] - 5s 492ms/step - loss: 1.9665 - accuracy: 0.3739 - val_loss: 2.0003 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 4/50
11/11 [==============================] - 5s 486ms/step - loss: 1.9015 - accuracy: 0.4522 - val_loss: 1.9396 - val_accuracy: 0.3158 - lr: 0.0010
Epoch 5/50
11/11 [==============================] - 6s 506ms/step - loss: 1.8356 - accuracy: 0.4812 - val_loss: 1.8792 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 6/50
11/11 [==============================] - 5s 480ms/step - loss: 1.7517 - accuracy: 0.5594 - val_loss: 1.8187 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 7/50
11/11 [==============================] - 5s 478ms/step - 

In [ ]:
labels = stacked_model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head(10)

,Image,target
0,508.jpg,kuchipudi
1,246.jpg,kathakali
2,473.jpg,kathakali
3,485.jpg,kuchipudi
4,128.jpg,bharatanatyam
5,410.jpg,kuchipudi
6,465.jpg,kathakali
7,196.jpg,kathakali
8,340.jpg,manipuri
9,467.jpg,kathakali


In [ ]:
 submission.to_csv('/content/drive/My Drive/dataset/stacking.csv', index=False)